# Sagar's attempt at cleaning all entries

In [1]:
import pandas as pd
import numpy as np

filepath = "./recipeitems-latest.json"

with open(filepath, 'rb') as f:
    data = f.readlines()
    
#remove trailing "\n" from lines
data = map(lambda x :x.rstrip(), data)
#put in good format:
data_json_str = '[' +','.join(data)+ ']'

#load into pandas
recipe_df = pd.read_json(data_json_str)

#get rid of extra columns
cleaned_recipes = recipe_df[['ingredients', 'name']].copy()
cleaned_recipes['after'] = np.nan
cleaned_recipes.head()

,ingredients,name,after
0,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...,Drop Biscuits and Sausage Gravy,NaN
1,12 whole Dinner Rolls Or Small Sandwich Buns (...,Hot Roast Beef Sandwiches,NaN
2,Dressing:\n1 tablespoon cumin seeds\n1/3 cup /...,Morrocan Carrot and Chickpea Salad,NaN
3,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...,Mixed Berry Shortcake,NaN
4,For each bowl: \na big dollop of Greek yogurt\...,Pomegranate Yogurt Bowl,NaN


In [2]:
#get number of recipes
cleaned_recipes.shape

(173278, 3)

In [3]:
#get food table
foods_df = pd.read_excel('./cofids.xls')
foods_df = foods_df.ix[2:]
#foods_df.head()

#get food names from food table
food_names = foods_df['Food Name']
#food_names.head()

#get food item from food names
##should be noted that the food_items list is not completely clean
food_items = []
for row in food_names:
    words = row.split(',')
    if words[0].lower() not in food_items:
        food_items.append(words[0].lower())

#add more food items using ingredients.csv
ingredients_df = pd.read_csv('./ingredients.csv')
ingredients = ingredients_df[ingredients_df.columns[0]]

more_food = []
for food in ingredients:
    more_food.append(food.lower().decode('utf-8'))

for food in more_food:
     if food not in food_items:
        food_items.append(food)

In [4]:
#number of unique ingredients
len(food_items)

1589

In [5]:
#http://stackoverflow.com/questions/4154961/find-substring-in-string-but-only-if-whole-words
import re

def string_found(string1, string2):
    if re.search(r"\b" + re.escape(string1) + r"\b", string2):
        return True
    return False

In [6]:
#function to clean the ingredients

def cleanIngredients(df, threadNum ,message = 'Complete!'):
    print str(threadNum) + ' starting now!'
    measurements = ['cup','cups','c','teaspoon','teaspoons','tsp','tablespoon','tablespoons','tbsp','pint','pints','bunch','bunches','dash','dashes','box','boxes','drizzle','loaf','loaves','dollop','clove','cloves','handful','handfuls','pinch','pinches','package','packages','stick','sticks','pound','pounds','g','mg','l','kg','ml','whole','can','cans','half','gallon','gallons','quart','quarts','qt','ounce','ounces','oz']
    cleaned_data = []
    complete_entries = 0 
    
    for index, row in df.iterrows():
        ingredients = []
        cleaned_ingr_test = []
        cleaned_ingredients = []
        ingr = row['ingredients']
        ingredients = ingr.split("\n")

        for recipe in ingredients:
            ingr_list = re.split('\W+\d+',recipe)
            for item in ingr_list:
                if item.isnumeric():
                    ind = ingr_list.index(item)
                    del ingr_list[ind]
            cleaned_ingr_test.append(" ".join(ingr_list)) 
            
        test_2 = []
        for ingredient in cleaned_ingr_test:
            ingr_list = ingredient.split()
            for m in measurements:
                if m in ingr_list:
                    del ingr_list[ingr_list.index(m)]
            test_2.append(" ".join(ingr_list))

        test_3 = []
        for ingredient in test_2:
            ingr_list = ingredient.split()
            for item in ingr_list:
                if item.isnumeric():
                    ind = ingr_list.index(item)
                    del ingr_list[ind]
            test_3.append(" ".join(ingr_list))
        
        for string in test_3:
            found = False
            matched_food = ''
            for food in food_items:
                if string_found(food.lower(),string.lower()):
                    found = True
                    if len(food) > len(matched_food):
                        matched_food = food.lower()
            if found:
                cleaned_ingredients.append(matched_food)
            else:    
                cleaned_ingredients.append(string.lower())

        cleaned_data.append('\n'.join(cleaned_ingredients))
        if index % 500 == 0:
            print 'Process ' + str(threadNum) + ' update: ' + str(complete_entries) 
        complete_entries = complete_entries + 1
    print 'Process ' + str(threadNum) + ' ' + str(message)
    return cleaned_data



In [ ]:
import multiprocessing as mp
pool = mp.Pool(5)
f = pool.apply_async(cleanIngredients, args = (cleaned_recipes[0:43319], 0,))
d = pool.apply_async(cleanIngredients, args = (cleaned_recipes[43320:86638], 1,))
s = pool.apply_async(cleanIngredients, args = (cleaned_recipes[86639:129957], 2,))
a = pool.apply_async(cleanIngredients, args = (cleaned_recipes[129958:173276], 3,))
b = pool.apply_async(cleanIngredients, args = (cleaned_recipes[173276:], 4,))
one = f.get()
two = d.get()
three = s.get()
four = a.get()
five = b.get()

0 starting now!
1 starting now!
2 starting now!
3 starting now!
4 starting now!
Process 0 update: 0
Process 4 Complete!


In [8]:
import multiprocessing as mp

start = 0
end = 43319
num_processes = 4
myProcesses = []
pool = mp.Pool(5)

while num_processes > 0:
    f = pool.apply_async(cleanIngredients, args = (cleaned_recipes[start:end], 4 - num_processes,) )
    myProcesses.append(f)
    start = end + 1
    end = end + 43319
    num_processes = num_processes - 1
    
f = pool.apply_async(cleanIngredients, args = (cleaned_recipes[start:], 4,) )
myProcesses.append(f)

print start
print end


173277
216595


In [9]:
len(myProcesses)

5

In [ ]:
cleaned_peices = [] * 5
cleaned_peices[0] = myProcesses[0].get()
cleaned_peices[1] = myProcesses[1].get()
cleaned_peices[2] = myProcesses[2].get()
cleaned_peices[3] = myProcesses[3].get()
cleaned_peices[4] = myProcesses[4].get()